# 1. Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
register = pd.read_csv("register.csv")
rooms = pd.read_csv("rooms.csv")

# 2. Cleaning

In [18]:
# Aggregate amounts
grouped = register.groupby(["legal_entity", "service_category", "room_id", "month"])["amount"].sum().reset_index()
joined = pd.merge(grouped, rooms, on="room_id", how="left")
joined.head()

,legal_entity,service_category,room_id,month,amount,building_id,room_area,build_year,building_floors,building_type,building_func,x_coord,y_coord
0,False,Dvinarė kaina,005f6379f5dc0e1,2018-11-01,0.74,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
1,False,Dvinarė kaina,005f6379f5dc0e1,2018-12-01,0.74,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
2,False,Dvinarė kaina,005f6379f5dc0e1,2019-01-01,0.63,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
3,False,Dvinarė kaina,005f6379f5dc0e1,2019-02-01,0.63,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
4,False,Dvinarė kaina,005f6379f5dc0e1,2019-03-01,0.63,0a425666b45790e,45.1,1968-01-01,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0


In [22]:
joined["amount"].corr(joined["x_coord"])

np.float64(0.0022077555074536473)

In [46]:
dropped = joined.drop(["room_id", "month", "building_id", "building_type"], axis=1)
dropped["build_year"] = pd.to_datetime(dropped["build_year"])
dropped["build_year"] = dropped["build_year"].dt.year
dropped.head()

,legal_entity,service_category,amount,room_area,build_year,building_floors,building_func,x_coord,y_coord
0,False,Dvinarė kaina,0.74,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
1,False,Dvinarė kaina,0.74,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
2,False,Dvinarė kaina,0.63,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
3,False,Dvinarė kaina,0.63,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0
4,False,Dvinarė kaina,0.63,45.1,1968.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456564.0,6199541.0


In [51]:
dropped["building_func"].unique()

array(['Gyvenamasis (trijų ir daugiau butų - daugiaaukštis pastatas)',
       nan, 'Administracinė', 'Gyvenamasis (individualus pastatas)',
       'Kita', 'Viešbučių', 'Kultūros', 'Mokslo', 'Gydymo', 'Sporto',
       'Gamybos', 'Komercinės paskirties', 'Prekybos', 'Transporto',
       'Religinės', 'Sandėliavimo', 'Maitinimo'], dtype=object)

In [52]:
X, y = dropped.drop("amount", axis=1), dropped["amount"]

In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [54]:
X_train.head(3)

,legal_entity,service_category,room_area,build_year,building_floors,building_func,x_coord,y_coord
3156563,False,K.v. skaitiklio aptarnavimas,44.47,1964.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,456587.0,6199852.0
6515808,False,Karštas vanduo,42.80,1962.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,457546.0,6199242.0
5719695,False,Karštas vanduo,67.76,1985.0,5.0,Gyvenamasis (trijų ir daugiau butų - daugiaauk...,453198.0,6197919.0


In [55]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ("imp", SimpleImputer()),
    ("scaler", StandardScaler())
])

int_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
])

ohe_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

ord_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ord", OrdinalEncoder())
])

num_cols = ["room_area", "x_coord", "y_coord"]
int_cols = ["build_year", "building_floors"]
ohe_cols = ["service_category", "building_func"]
ord_cols = ["legal_entity"]

processor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("int", int_pipeline, int_cols),
    ("ohe", ohe_pipeline, ohe_cols),
    ("ord", ord_pipeline, ord_cols)
])

In [56]:
X_train_proc = processor.fit_transform(X_train)

MemoryError: Unable to allocate 1.70 GiB for an array with shape (9483053, 24) and data type float64